In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision.utils import save_image
%run datasetgenerator.ipynb

In [ ]:
class VAE(nn.Module):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(1800, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, latent_dim * 2)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1800),
            nn.Sigmoid()
        )
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def forward(self, x):
        z_params = self.encoder(x)
        mu, logvar = z_params.chunk(2, dim=1)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar


In [ ]:
# Hyperparameters
latent_dim = 20
batch_size = 64
learning_rate = 1e-3
epochs = 50

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VAE(latent_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
number_of_survive_samples = 2000
number_of_generated_samples = 20000
data, _ = create_dataset(number_of_survive_samples, number_of_generated_samples)
data = torch.FloatTensor(data).view(-1, 1800).to(device)
dataset = TensorDataset(data)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
#Loss Function
def vae_loss(recon_x, x, mu, logvar):
    BCE = nn.BCELoss(reduction='sum')
    reconstruction_loss = BCE(recon_x, x)
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return reconstruction_loss + kl_divergence


In [ ]:
for epoch in range(epochs):
    total_loss = 0
    for batch in dataloader:
        batch = batch[0]
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(batch)
        loss = vae_loss(recon_batch, batch, mu, logvar)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{epochs}] Loss: {total_loss / len(dataset)}")


In [ ]:
with torch.no_grad():
    z = torch.randn(number_of_survive_samples, latent_dim).to(device)
    generated_samples = model.decoder(z).cpu().view(-1, 30, 30)


In [ ]:
plt.imshow(generated_samples[0], cmap= 'gray')
plt.show()